<a href="https://colab.research.google.com/github/yuping322/Ashare/blob/main/ComfyUICivitai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://civitai.com/user/Legendaer">
  <img src="https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/4111e719-a2a7-494b-9102-abc95cb9970d/original=true/4111e719-a2a7-494b-9102-abc95cb9970d.jpeg"
       alt="avatar"
       height="100"
       style="border-radius: 50%;" />
</a>

 _ComfyUI Colab Notebook_

 ❗ Colab Pro is recommended.

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. You need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. You need to paste you Civit AI API Key to download certain models.
4. Run the **`ComfyUI Setup`** code cell
6. Follow the URL link **`https://xxxxxxx.loca.lt`** to launch the app. The password is above the link. Or Use Ngrok

_Notes_:
* Try pressing **`F5`** if the ComfyUI Manager is not visible.

| |Civitai|
|:--|:-:|
| 🏠 **Profile** | <a href="https://civitai.com/user/Legendaer"> <img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fshop.civitai.com%2Fcdn%2Fshop%2Ffiles%2Flogo.png%3Fcrop%3Dcenter%26height%3D2048%26v%3D1691928374%26width%3D2048&f=1&nofb=1&ipt=e2090a4c61b579bff8f823bdb7a5597336c651feabd00cb8388b160cf12374c6&ipo=images" width="100"/> </a>|
| 📘 **Article** | <a href="https://civitai.com/articles/8503/"> <img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fshop.civitai.com%2Fcdn%2Fshop%2Ffiles%2Flogo.png%3Fcrop%3Dcenter%26height%3D2048%26v%3D1691928374%26width%3D2048&f=1&nofb=1&ipt=e2090a4c61b579bff8f823bdb7a5597336c651feabd00cb8388b160cf12374c6&ipo=images" width="100"/> </a> |


In [ ]:
#@title 🚩 ComfyUI Setup
#@markdown ### ▶️ Colab Secret Names (Optional):
Secret_NGROK = '' #@param {type: "string"}
Secret_CIVITAI = '' #@param {type: "string"}
#@markdown ### ▶️ CivitAI API Key (Required for certain models):
Civitai_API_KEY = '' #@param {type: "string"}
#@markdown ### ▶️ NGROK API Key (if using Ngrok):
NGROK_API_KEY='' #@param {type: "string"}
use_local_storage = True #@param {type:"boolean"}
output_path = 'COMFY' #@param {type:"string"}

#@markdown ## Only check the models you are going to use:
#@markdown ### SD 3.5 models
SD35L = False #@param{type: "boolean"}
SD35M = False #@param{type: "boolean"}
#@markdown ### Flux models
Flux1_dev = False #@param{type: "boolean"}

#@markdown ### SDXL models

SDXL_1 = False #@param{type: "boolean"}
JuggernautXL_v8 = False #@param{type: "boolean"}
Pony_Diffusion_XL_v6 = False #@param{type: "boolean"}
Perfect_Pony_XL = True #@param{type: "boolean"}

#@markdown ### SD 1.5 models

Dreamshaper = False #@param{type: "boolean"}

#@markdown ### Other models
SDXLRefiner = True #@param{type: "boolean"}
AnimateDiff = False #@param{type: "boolean"}

#@markdown ### ▶️ Custom LoRA URL (Optional)
Custom_LoRA_URL = "https://civitai.com/api/download/models/370153" #@param {type: "string"}
Custom_LoRA_Name = "ZeldaXL.safetensors" #@param {type: "string"}
#@markdown ### ▶️ Custom Model URL (Optional)
Custom_Model_URL = "https://civitai.com/api/download/models/997031" #@param {type: "string"}
Custom_Model_Name = "SmoothMix.safetensors" #@param {type: "string"}

#@markdown ### ControlNet models:
SD_1_5_ControlNet_models = False #@param{type: "boolean"}
SDXL_ControlNet_models = True #@param{type: "boolean"}
IP_Adapter_models = True #@param{type: "boolean"}

#@markdown ### ControlNet models:
clip_l = True #@param{type: "boolean"}
t5xxl = False #@param{type: "boolean"}
clip_vision_g = False #@param{type: "boolean"}
google_t5_v1 = False #@param{type: "boolean"}

#@markdown ### ▶️ Extra ComfyUI arguments
Extra_arguments = '--disable-smart-memory --port 8188' #@param {type: "string"}

Clear_Log = True #@param{type: "boolean"}

#####################
import subprocess
import threading
import time
import socket
import urllib.request
import os
import glob

def link_files(source, dest):
  if not os.path.exists(source):
    !mkdir -p {source}
  if not os.path.exists(dest):
    !mkdir -p {dest}
  model_files = glob.glob(source + '/*')
  %cd {dest}
  for f in model_files:
    print(f'Linking model {f} in {dest}')
    !ln -s {f}

def initStorage(output_path, app_local_path):
  if use_local_storage:
    # Use local storage
    app_storage_path = "/content/local_storage"
    models_storage_path = f"{app_storage_path}/models"
    image_output_path = f"{app_storage_path}/outputs"
    image_input_path = f"{app_storage_path}/inputs"

    # Create local directories
    !mkdir -p {app_storage_path}
    !mkdir -p {models_storage_path}
    !mkdir -p {image_output_path}
    !mkdir -p {image_input_path}
  else:
    # Use Google Drive storage
    from google.colab import drive
    drive.mount('/content/drive')

    app_storage_path = f"/content/drive/MyDrive/{output_path}"
    models_storage_path = f"{app_storage_path}/models"
    image_output_path = f"{app_storage_path}/outputs"
    image_input_path = f"{app_storage_path}/inputs"

    # Create Google Drive directories
    !mkdir -p {app_storage_path}
    !mkdir -p {models_storage_path}
    !mkdir -p {image_output_path}
    !mkdir -p {image_input_path}

  # Set up model directories
  model_path_map = {
      f'{models_storage_path}/embeddings': f'{app_local_path}/models/embeddings',
      f'{models_storage_path}/animatediff': f'{app_local_path}/models/animatediff_models',
      f'{models_storage_path}/clip': f'{app_local_path}/models/clip',
      f'{models_storage_path}/clip_vision': f'{app_local_path}/models/clip_vision',
      f'{models_storage_path}/ControlNet': f'{app_local_path}/models/controlnet',
      f'{models_storage_path}/ESRGAN': f'{app_local_path}/models/upscale_models',
      f'{models_storage_path}/hypernetworks': f'{app_local_path}/models/hypernetworks',
      f'{models_storage_path}/insightface': f'{app_local_path}/models/insightface',
      f'{models_storage_path}/instantid': f'{app_local_path}/models/instantid',
      f'{models_storage_path}/layer_model': f'{app_local_path}/models/layer_model',
      f'{models_storage_path}/LDSR': f'{app_local_path}/models/LDSR',
      f'{models_storage_path}/liveportrait': f'{app_local_path}/models/liveportrait',
      f'{models_storage_path}/Lora': f'{app_local_path}/models/loras',
      f'{models_storage_path}/reactor': f'{app_local_path}/models/reactor',
      f'{models_storage_path}/Stable-diffusion': f'{app_local_path}/models/checkpoints',
      f'{models_storage_path}/unet': f'{app_local_path}/models/unet',
      f'{models_storage_path}/VAE': f'{app_local_path}/models/vae',
      f'{models_storage_path}/VAE-approx': f'{app_local_path}/models/vae',
      f'{models_storage_path}/xlabs/loras': f'{app_local_path}/models/xlabs/loras',
      f'{models_storage_path}/xlabs/controlnets': f'{app_local_path}/models/xlabs/controlnets',
  }

  # Create symbolic links for all models
  for model_path in model_path_map:
    link_files(model_path, model_path_map[model_path])

  return image_output_path, image_input_path

def iframe_thread(port):
  while True:
      time.sleep(1)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("Tunnel Password:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')

def clear():
    from IPython.display import clear_output; return clear_output()

def downloadModel(url, rename=None, token=None):
    # Hugging Face model download
    if 'huggingface.co' in url:
        if rename:
            filename = rename
        else:
            filename = url.split('/')[-1].split('?')[0]  # Remove query parameters
        print(f"Downloading Hugging Face model: {filename}")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}

    # Civitai model download
    elif 'civitai.com' in url:
        filename = rename or url.split('/')[-1].split('?')[0]
        url_with_token = f"{url}?token={token}" if token else url
        print(f"Downloading Civitai model: {filename}")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url_with_token} -o {filename}
    else:
        print("Unsupported URL format.")

def install_dependencies(repo_name):
  requirements_path = f"/content/ComfyUI/custom_nodes/{repo_name}/requirements.txt"
  if os.path.exists(requirements_path):
      print(f"Installing dependencies for {repo_name}...")
      subprocess.run(['pip', 'install', '-r', requirements_path], check=True)
  else:
      print(f"No requirements.txt found for {repo_name}. Skipping dependency installation.")

def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  repo_name = url.split('/')[-1]  # Extract repo name from the URL
  !git clone {url} --recursive
  install_dependencies(repo_name)

def install_comfyui_impact():
    %cd /content/ComfyUI/custom_nodes

    # Clone the repository manually
    !git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git

    # Change directory to the cloned repository
    %cd ComfyUI-Impact-Pack

    # Install dependencies from requirements.txt
    print("Installing dependencies from requirements.txt...")
    !pip install -r requirements.txt

    # Install the 'ultralytics' library, which is missing and needed for YOLO
    print("Installing ultralytics for YOLO...")
    !pip install ultralytics

    # Run the install.py script to complete the installation
    print("Running install.py...")
    !python install.py

def install_custom_nodes():
  %cd {customNodeRoot}
  install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager')
  install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')
  install_custom_node('https://github.com/jags111/efficiency-nodes-comfyui')
  install_custom_node('https://github.com/ssitu/ComfyUI_UltimateSDUpscale')
  install_custom_node('https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes')
  install_custom_node('https://github.com/Fannovel16/comfyui_controlnet_aux')
  install_custom_node('https://github.com/pythongosssss/ComfyUI-Custom-Scripts')
  install_custom_node('https://github.com/rgthree/rgthree-comfy')
  install_custom_node('https://github.com/Acly/comfyui-inpaint-nodes')
  install_custom_node('https://github.com/ltdrdata/ComfyUI-Inspire-Pack')
  install_custom_node('https://github.com/WASasquatch/was-node-suite-comfyui')
  install_custom_node('https://github.com/XLabs-AI/x-flux-comfyui')
  install_custom_node('https://github.com/pythongosssss/ComfyUI-WD14-Tagger')
  install_custom_node('https://github.com/cubiq/ComfyUI_IPAdapter_plus')
  install_custom_node('https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved')
  install_custom_node('https://github.com/civitai/civitai_comfy_nodes')
  install_custom_node('https://github.com/cubiq/ComfyUI_essentials')
  install_custom_node('https://github.com/BadCafeCode/masquerade-nodes-comfyui')
  install_custom_node('https://github.com/sipherxyz/comfyui-art-venture')
  install_custom_node('https://github.com/twri/sdxl_prompt_styler')
  install_custom_node('https://github.com/cubiq/ComfyUI_essentials')
  install_custom_node('https://github.com/giriss/comfy-image-saver')
  install_custom_node('https://github.com/crystian/ComfyUI-Crystools')
  install_custom_node('https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite')
  install_custom_node('https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet')
  install_custom_node('https://github.com/Fannovel16/ComfyUI-Frame-Interpolation')
  install_comfyui_impact()

def install_Models():
  %cd {ComfyUIRoot}/models/checkpoints
  print('⏱ Downloading models ...')
  # Flux
  if Flux1_dev:
    downloadModel('https://huggingface.co/Comfy-Org/flux1-dev/resolve/main/flux1-dev-fp8.safetensors')
  #SD3.5
  if SD35L:
    downloadModel('https://civitai.com/api/download/models/983309',rename='SD35L.safetensors',token=Civitai_API_KEY)
  if SD35M:
    downloadModel('https://civitai.com/api/download/models/1003708',rename='SD35M.safetensors',token=Civitai_API_KEY)

  # SDXL/Pony
  if SDXLRefiner:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')
  if SDXL_1:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
  if JuggernautXL_v8:
    downloadModel('https://civitai.com/api/download/models/288982')
  if Pony_Diffusion_XL_v6:
    downloadModel('https://huggingface.co/Magamanny/Pony-Diffusion-V6-XL/resolve/main/ponyDiffusionV6XL_v6StartWithThisOne.safetensors')
  if Perfect_Pony_XL:
    #downloadModel('https://civitai.com/api/download/models/828380',rename="Perfect_Pony_XL.safetensors",token=Civitai_API_KEY) #V3
    downloadModel('https://civitai.com/api/download/models/1047139',rename="Perfect_Pony_XL.safetensors",token=Civitai_API_KEY) #V6

  #SD1.5
  if Dreamshaper:
    downloadModel('https://civitai.com/api/download/models/128713',rename="DreamShaper.safetensors",token=Civitai_API_KEY)

  if Custom_Model_URL and Custom_Model_Name:
    print(f"Downloading custom model: {Custom_Model_Name}")
    downloadModel(Custom_Model_URL, rename=Custom_Model_Name, token=Civitai_API_KEY)


def install_LoRAModels():
  %cd {ComfyUIRoot}/models/loras

  if Custom_LoRA_URL and Custom_LoRA_Name:
    print(f"Downloading custom LoRA model: {Custom_LoRA_Name}")
    downloadModel(Custom_LoRA_URL, rename=Custom_LoRA_Name, token=Civitai_API_KEY)

def install_Embeddings():
  %cd {ComfyUIRoot}/models/embeddings
  downloadModel('https://civitai.com/api/download/models/9208',rename="easynegative.safetensors")
  downloadModel('https://civitai.com/api/download/models/930629',rename="deep_negative_pony.safetensors")

def install_VAE(vae_to_install):
  %cd {ComfyUIRoot}/models/vae
  downloadModel(str(vae_to_install))

def install_CLIP():
  %cd {ComfyUIRoot}/models/clip
  print(f"Downloading Clip Models")
  if(clip_l):
    downloadModel('https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors')
  if(t5xxl):
   downloadModel('https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors')
  if(clip_vision_g):
   downloadModel('https://huggingface.co/stabilityai/control-lora/resolve/main/revision/clip_vision_g.safetensors')
  if(google_t5_v1):
    downloadModel('https://huggingface.co/mcmonkey/google_t5-v1_1-xxl_encoderonly/resolve/main/t5xxl_fp8_e4m3fn.safetensors')

def install_Upscaler():
  %cd {ComfyUIRoot}/models/upscale_models
  downloadModel('https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth')
  downloadModel('https://huggingface.co/FacehugmanIII/4x_foolhardy_Remacri/resolve/main/4x_foolhardy_Remacri.pth')

def install_AnimateDiff():
  if AnimateDiff:
    %cd {ComfyUIRoot}/models/animatediff_models
    downloadModel('https://huggingface.co/wangfuyun/AnimateLCM/resolve/main/AnimateLCM_sd15_t2v.ckpt')
    %cd {ComfyUIRoot}/models/loras
    downloadModel('https://huggingface.co/wangfuyun/AnimateLCM/resolve/main/AnimateLCM_sd15_t2v_lora.safetensors')
    install_VAE('https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors')

def install_ControlNetModels():
  %cd {ComfyUIRoot}/models/controlnet
  if SD_1_5_ControlNet_models:
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')


  if SDXL_ControlNet_models:
    downloadModel('https://huggingface.co/xinsir/controlnet-openpose-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors', rename='diffusion_xl_openpose.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')

  if IP_Adapter_models:
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin')
    !pip install insightface

def ngrok_thread(port):
  PASSWORD='googlecolab'
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  ngrok.set_auth_token(NGROK_API_KEY)
  http_tunnel = ngrok.connect(port, auth=f"{PASSWORD}:{PASSWORD}")
  print(f'Username/password: {PASSWORD}')
  print(f'ngrok public URL: {http_tunnel.public_url}')

#################################


!pkill -f 'python main.py'  # Kill any existing ComfyUI instances
!rm -rf /content/ComfyUI/logs/*  # Clear old logs

# Get secrets
from google.colab import userdata
try:
  NGROK_API_KEY = userdata.get(Secret_NGROK)
  print('Used NGROK secret.')
except:
  NGROK_API_KEY = None
try:
  Civitai_API_KEY = userdata.get(Secret_CIVITAI)
  print('Used Civitai secret.')
except:
  Civitai_API_KEY = None

# Define folders
root = '/content/'
ComfyUIRoot = os.path.join(root, 'ComfyUI')
customNodeRoot = os.path.join(ComfyUIRoot, 'custom_nodes')


# Install ComfyUI
!pip install --upgrade torch torchvision
%cd {root}
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd {ComfyUIRoot}
!pip install -r requirements.txt

# Initialize storage and get paths
image_output_path, image_input_path = initStorage(output_path, ComfyUIRoot)


!apt-get -y install -qq aria2

# ngrok
if NGROK_API_KEY:
  %pip install pyngrok
  from pyngrok import ngrok

# Install models
install_Models()
install_LoRAModels()
install_Embeddings()
install_ControlNetModels()
install_AnimateDiff()
install_Upscaler()
install_CLIP()

# Install custom nodes
install_custom_nodes()

# localtunnel
!npm install -g localtunnel
if Clear_Log:
  clear()
if NGROK_API_KEY:
  port = 8188
  threading.Thread(target=ngrok_thread, daemon=True, args=(port,)).start()
else:
  # Local tunnel
  !npm install -g localtunnel
  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server --output-directory {image_output_path} --input-directory {image_input_path} {Extra_arguments}




changed 22 packages, and audited 23 packages in 963ms

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
/content/ComfyUI
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2024-11-22 03:10:33.721319
** Platform: Linux
** Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** Log path: /content/ComfyUI/comfyui.log

Prestartup times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/rgthree-comfy
   1.2 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Total VRAM 15102 MB, total RAM 12979 MB
pytorch version: 2.5.1+cu121
Set vram state to: NORMAL_VRAM
Disabling smart memory management
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using pytorch cross attention
**

In [ ]:
#@title 📂 Zip Outputs
#@markdown ### ▶️ Zipping:
use_google_drive= False #@param {type: "boolean"}
zip_prefix = 'ComfyUI_' #@param {type: "string"}
zip_subfolder = '' #@param {type: "string"}


import os
import zipfile
import glob
from datetime import datetime

def zip_and_delete_pngs(prefix, use_google_drive=True, subfolder=None):
    """
    Zips all PNG files with a specified prefix and deletes them afterwards.
    Works with both Google Drive and local storage in ComfyUI environment.

    Returns:
        str: Path to the created zip file
    """
    try:
        # Determine base path based on storage type
        if use_google_drive:
            base_path = '/content/drive/MyDrive/COMFY/outputs'
        else:
            base_path = '/content/comfy_local/outputs'

        # Add subfolder if specified
        if subfolder:
            base_path = os.path.join(base_path, subfolder)

        # Create base directory if it doesn't exist
        os.makedirs(base_path, exist_ok=True)

        # Generate zip filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        zip_filename = f'{prefix}files_{timestamp}.zip'
        zip_path = os.path.join(base_path, zip_filename)

        # Find all matching PNG files
        search_pattern = os.path.join(base_path, f'{prefix}*.png')
        png_files = glob.glob(search_pattern)

        if not png_files:
            print(f"No PNG files found with prefix '{prefix}' in {base_path}")
            return None

        # Create zip file
        print(f"Found {len(png_files)} files to process")
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for png_file in png_files:
                # Get relative path for zip storage
                arcname = os.path.basename(png_file)

                # Add to zip
                zipf.write(png_file, arcname)
                print(f"Added {arcname} to zip")

                # Delete original file
                os.remove(png_file)
                print(f"Deleted {arcname}")

        print(f"\nProcess completed successfully!")
        print(f"Zip file created at: {zip_path}")
        return zip_path

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Execute
zip_and_delete_pngs(zip_prefix, use_google_drive, zip_subfolder)


## 📅 Changelog
|dd.mm.yy|Change|
|:-|:-|
|`25/10/24`| Created Notebook|
|`26/10/24`| Added Civitai API Key|
|`28/10/24`| Added custom LoRA and Checkpoint download|
|`30/10/24`| Added output zipping|
|`31/10/24`| Added Upscalers and some Improvements|
|`09/11/24`| Added CLIP and SD 3.5 Medium|
|`15/11/24`| Added Ngrok support and Colab secrets for API Keys|
|`18/11/24`| Basic Improvements|